In [3]:
import chess.pgn
import pandas as pd
import os
import re
import json

def extract_blunders(pgn_file):
    blunders = []

    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            headers = game.headers
            node = game
            move_number = 0

            while node.variations:
                next_node = node.variation(0)
                move_number += 1

                comment = next_node.comment
                eval_match = re.search(r'\[%eval (-?\d+\.\d+)\]', comment)
                if eval_match:
                    eval_score = float(eval_match.group(1))
                else:
                    eval_score = None

                if "Blunder" in comment or "blunder" in comment:
                    before_blunder = node.board().fen()
                    after_blunder = next_node.board().fen()

                    better_move_match = re.search(r'\{ Blunder. (.*?) was best\.', comment)
                    if better_move_match:
                        better_move = better_move_match.group(1)
                    else:
                        better_move = None

                    blunder_info = {
                        "Event": headers.get("Event", "Unknown"),
                        "Site": headers.get("Site", "Unknown"),
                        "Date": headers.get("Date", "Unknown"),
                        "Round": headers.get("Round", "Unknown"),
                        "White Player": headers.get("White", "Unknown"),
                        "Black Player": headers.get("Black", "Unknown"),
                        "Result": headers.get("Result", "Unknown"),
                        "UTCDate": headers.get("UTCDate", "Unknown"),
                        "UTCTime": headers.get("UTCTime", "Unknown"),
                        "Variant": headers.get("Variant", "Standard"),
                        "ECO": headers.get("ECO", "Unknown"),
                        "Opening": headers.get("Opening", "Unknown"),
                        "Move Number": move_number,
                        "Player": headers.get("White", "Unknown") if move_number % 2 == 1 else headers.get("Black", "Unknown"),
                        "Blunder Move": node.board().san(next_node.move),
                        "Better Move": better_move,
                        "Comment": comment,
                        "Evaluation": eval_score,
                        "Position Before Blunder": before_blunder,
                        "Position After Blunder": after_blunder,
                    }
                    blunders.append(blunder_info)

                node = next_node

    return blunders

def process_pgn_files(pgn_directory, output_file):
    all_blunders = []

    for filename in os.listdir(pgn_directory):
        if filename.endswith(".pgn"):
            pgn_file = os.path.join(pgn_directory, filename)
            blunders = extract_blunders(pgn_file)
            all_blunders.extend(blunders)

    with open(output_file, 'w') as json_file:
        json.dump(all_blunders, json_file, indent=4)
    print(f"Blunders saved to {output_file}")

pgn_directory = "Analysis_pgns_small"
output_file = "blunders.json"
process_pgn_files(pgn_directory, output_file)

Blunders saved to blunders.json


In [5]:
import os
import chess
import chess.pgn
import json
import io

def parse_pgn(file_path):
    with open(file_path, "r") as f:
        pgn_text = f.read()
    return pgn_text

def get_blunders_from_game(game, game_number):
    blunders = []
    board = game.board()
    previous_eval = None

    for move_number, move in enumerate(game.mainline_moves(), start=1):
        fen_before = board.fen()
        board.push(move)
        fen_after = board.fen()
        node = game.next()
        if node is None:
            break
        comment = node.comment
        if "Blunder" in comment:
            current_eval = node.eval()
            blunder_info = {
                "game_number": game_number,
                "white": game.headers["White"],
                "black": game.headers["Black"],
                "move_number": move_number,
                "move": board.peek().uci(),
                "player": "White" if board.turn else "Black",
                "fen_before": fen_before,
                "fen_after": fen_after,
                "comment": comment,
                "previous_eval": previous_eval,
                "current_eval": current_eval,
                "blunder_change": current_eval - previous_eval if previous_eval is not None else None,
                "opening": game.headers.get("Opening", ""),
                "eco": game.headers.get("ECO", ""),
                "round": game.headers.get("Round", ""),
                "event": game.headers.get("Event", ""),
                "site": game.headers.get("Site", ""),
                "date": game.headers.get("Date", "")
            }
            blunders.append(blunder_info)
        previous_eval = node.eval()

    return blunders

def process_pgn_files(directory_path):
    all_blunders = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pgn"):
            file_path = os.path.join(directory_path, file_name)
            pgn_text = parse_pgn(file_path)
            pgn = io.StringIO(pgn_text)
            game_counter = 1

            while True:
                game = chess.pgn.read_game(pgn)
                if game is None:
                    break

                blunders = get_blunders_from_game(game, game_counter)
                all_blunders.extend(blunders)
                game_counter += 1

    return all_blunders

def save_to_json(data, file_path):
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)

if __name__ == "__main__":
    pgn_directory_path = "Analysis_pgns_small"  # Change to your directory
    json_output_path = "blundersnew.json"

    all_blunders = process_pgn_files(pgn_directory_path)
    save_to_json(all_blunders, json_output_path)
    print(f"Blunders have been saved to {json_output_path}")

Blunders have been saved to blundersnew.json


In [6]:
import os
import re
import json
import chess.pgn
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

def parse_pgn(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def extract_blunders(pgn_file):
    blunders = []

    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            headers = game.headers
            node = game
            move_number = 0

            while node.variations:
                next_node = node.variation(0)
                move_number += 1

                comment = next_node.comment
                eval_match = re.search(r'\[%eval (-?\d+\.\d+)\]', comment)
                if eval_match:
                    eval_score = float(eval_match.group(1))
                else:
                    eval_score = None

                if re.search(r'\b[Bb]lunder\b', comment):
                    before_blunder = node.board().fen()
                    after_blunder = next_node.board().fen()

                    better_move_match = re.search(r'\{.*?Blunder.*?(\b\w+\b) was best\.', comment)
                    if better_move_match:
                        better_move = better_move_match.group(1)
                    else:
                        better_move = None

                    blunder_info = {
                        "Event": headers.get("Event", "Unknown"),
                        "Site": headers.get("Site", "Unknown"),
                        "Date": headers.get("Date", "Unknown"),
                        "Round": headers.get("Round", "Unknown"),
                        "White Player": headers.get("White", "Unknown"),
                        "Black Player": headers.get("Black", "Unknown"),
                        "Result": headers.get("Result", "Unknown"),
                        "UTCDate": headers.get("UTCDate", "Unknown"),
                        "UTCTime": headers.get("UTCTime", "Unknown"),
                        "Variant": headers.get("Variant", "Standard"),
                        "ECO": headers.get("ECO", "Unknown"),
                        "Opening": headers.get("Opening", "Unknown"),
                        "Move Number": move_number,
                        "Player": headers.get("White", "Unknown") if move_number % 2 == 1 else headers.get("Black", "Unknown"),
                        "Blunder Move": node.board().san(next_node.move),
                        "Better Move": better_move,
                        "Comment": comment,
                        "Evaluation": eval_score,
                        "Position Before Blunder": before_blunder,
                        "Position After Blunder": after_blunder,
                    }
                    blunders.append(blunder_info)

                node = next_node

    return blunders

def process_pgn_files(directory_path):
    all_blunders = []
    files = [os.path.join(directory_path, file_name) for file_name in os.listdir(directory_path) if file_name.endswith(".pgn")]

    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(extract_blunders, file): file for file in files}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing PGN Files"):
            blunders = future.result()
            all_blunders.extend(blunders)

    return all_blunders

def save_to_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    pgn_directory_path = "Analysis_pgns_small"
    json_output_path = "blunders.json"

    all_blunders = process_pgn_files(pgn_directory_path)
    save_to_json(all_blunders, json_output_path)
    print(f"Blunders have been saved to {json_output_path}")

Process SpawnProcess-2:
Traceback (most recent call last):
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'extract_blunders' on <module '__main__' (built-in)>
Process SpawnProcess-1:
Traceback (most recent call last):
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/alankritverma/miniconda3/envs/CSS

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [8]:
import os
import re
import json
import chess.pgn

def parse_pgn(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def extract_blunders(pgn_file):
    blunders = []

    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            headers = game.headers
            node = game
            move_number = 0

            while node.variations:
                next_node = node.variation(0)
                move_number += 1

                comment = next_node.comment
                eval_match = re.search(r'\[%eval (-?\d+\.\d+)\]', comment)
                if eval_match:
                    eval_score = float(eval_match.group(1))
                else:
                    eval_score = None

                if re.search(r'\b[Bb]lunder\b', comment):
                    before_blunder = node.board().fen()
                    after_blunder = next_node.board().fen()

                    better_move_match = re.search(r'\{.*?Blunder.*?(\b\w+\b) was best\.', comment)
                    if better_move_match:
                        better_move = better_move_match.group(1)
                    else:
                        better_move = None

                    blunder_info = {
                        "Event": headers.get("Event", "Unknown"),
                        "Site": headers.get("Site", "Unknown"),
                        "Date": headers.get("Date", "Unknown"),
                        "Round": headers.get("Round", "Unknown"),
                        "White Player": headers.get("White", "Unknown"),
                        "Black Player": headers.get("Black", "Unknown"),
                        "Result": headers.get("Result", "Unknown"),
                        "UTCDate": headers.get("UTCDate", "Unknown"),
                        "UTCTime": headers.get("UTCTime", "Unknown"),
                        "Variant": headers.get("Variant", "Standard"),
                        "ECO": headers.get("ECO", "Unknown"),
                        "Opening": headers.get("Opening", "Unknown"),
                        "Move Number": move_number,
                        "Player": headers.get("White", "Unknown") if move_number % 2 == 1 else headers.get("Black", "Unknown"),
                        "Blunder Move": node.board().san(next_node.move),
                        "Better Move": better_move,
                        "Comment": comment,
                        "Evaluation": eval_score,
                        "Position Before Blunder": before_blunder,
                        "Position After Blunder": after_blunder,
                    }
                    blunders.append(blunder_info)

                node = next_node

    return blunders

def process_pgn_files(directory_path):
    all_blunders = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pgn"):
            file_path = os.path.join(directory_path, file_name)
            blunders = extract_blunders(file_path)
            all_blunders.extend(blunders)

    return all_blunders

def save_to_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    pgn_directory_path = "Analysis_pgns_small"
    json_output_path = "blundersss.json"

    all_blunders = process_pgn_files(pgn_directory_path)
    save_to_json(all_blunders, json_output_path)
    print(f"Blunders have been saved to {json_output_path}")

Blunders have been saved to blundersss.json


In [9]:
import os
import re
import json
import chess.pgn

def parse_pgn(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def extract_blunders(pgn_file):
    blunders = []

    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            headers = game.headers
            node = game
            move_number = 0

            while node.variations:
                next_node = node.variation(0)
                move_number += 1

                comment = next_node.comment
                eval_match = re.search(r'\[%eval (-?\d+\.\d+)\]', comment)
                eval_match_prev = re.search(r'\[%eval (-?\d+\.\d+)\]', node.comment)
                if eval_match and eval_match_prev:
                    eval_score = float(eval_match.group(1))
                    eval_score_prev = float(eval_match_prev.group(1))
                else:
                    eval_score = None
                    eval_score_prev = None

                if re.search(r'\b[Bb]lunder\b', comment):
                    before_blunder = node.board().fen()
                    after_blunder = next_node.board().fen()

                    better_move_match = re.search(r'\{.*?Blunder.*?(\b\w+\b) was best\.', comment)
                    if better_move_match:
                        better_move = better_move_match.group(1)
                    else:
                        better_move = None

                    eval_diff = None
                    if eval_score is not None and eval_score_prev is not None:
                        eval_diff = eval_score - eval_score_prev

                    blunder_info = {
                        "Event": headers.get("Event", "Unknown"),
                        "Site": headers.get("Site", "Unknown"),
                        "Date": headers.get("Date", "Unknown"),
                        "Round": headers.get("Round", "Unknown"),
                        "White Player": headers.get("White", "Unknown"),
                        "Black Player": headers.get("Black", "Unknown"),
                        "Result": headers.get("Result", "Unknown"),
                        "UTCDate": headers.get("UTCDate", "Unknown"),
                        "UTCTime": headers.get("UTCTime", "Unknown"),
                        "Variant": headers.get("Variant", "Standard"),
                        "ECO": headers.get("ECO", "Unknown"),
                        "Opening": headers.get("Opening", "Unknown"),
                        "Move Number": move_number,
                        "Player": headers.get("White", "Unknown") if move_number % 2 == 1 else headers.get("Black", "Unknown"),
                        "Color": "White" if move_number % 2 == 1 else "Black",
                        "Blunder Move": node.board().san(next_node.move),
                        "Better Move": better_move,
                        "Comment": comment,
                        "Evaluation Before": eval_score_prev,
                        "Evaluation After": eval_score,
                        "Evaluation Difference": eval_diff,
                        "Position Before Blunder": before_blunder,
                        "Position After Blunder": after_blunder,
                    }
                    blunders.append(blunder_info)

                node = next_node

    return blunders

def process_pgn_files(directory_path):
    all_blunders = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pgn"):
            file_path = os.path.join(directory_path, file_name)
            blunders = extract_blunders(file_path)
            all_blunders.extend(blunders)

    return all_blunders

def save_to_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    pgn_directory_path = "Analysis_pgns_small"
    json_output_path = "blunders123.json"

    all_blunders = process_pgn_files(pgn_directory_path)
    save_to_json(all_blunders, json_output_path)
    print(f"Blunders have been saved to {json_output_path}")

Blunders have been saved to blunders123.json


In [10]:
import os
import re
import json
import chess.pgn

def parse_pgn(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def extract_blunders(pgn_file):
    blunders = []

    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            headers = game.headers
            node = game
            move_number = 0

            while node.variations:
                next_node = node.variation(0)
                move_number += 1

                comment = next_node.comment
                eval_match = re.search(r'\[%eval (-?\d+\.\d+)\]', comment)
                eval_match_prev = re.search(r'\[%eval (-?\d+\.\d+)\]', node.comment)
                if eval_match and eval_match_prev:
                    eval_score = float(eval_match.group(1))
                    eval_score_prev = float(eval_match_prev.group(1))
                else:
                    eval_score = None
                    eval_score_prev = None

                if re.search(r'\b[Bb]lunder\b', comment):
                    before_blunder = node.board().fen()
                    after_blunder = next_node.board().fen()

                    better_move_match = re.search(r'Blunder\.\s*(.*?)\s*was best\.', comment)
                    if better_move_match:
                        better_move = better_move_match.group(1)
                    else:
                        better_move = None

                    eval_diff = None
                    if eval_score is not None and eval_score_prev is not None:
                        eval_diff = eval_score - eval_score_prev

                    blunder_info = {
                        "Event": headers.get("Event", "Unknown"),
                        "Site": headers.get("Site", "Unknown"),
                        "Date": headers.get("Date", "Unknown"),
                        "Round": headers.get("Round", "Unknown"),
                        "White Player": headers.get("White", "Unknown"),
                        "Black Player": headers.get("Black", "Unknown"),
                        "Result": headers.get("Result", "Unknown"),
                        "UTCDate": headers.get("UTCDate", "Unknown"),
                        "UTCTime": headers.get("UTCTime", "Unknown"),
                        "Variant": headers.get("Variant", "Standard"),
                        "ECO": headers.get("ECO", "Unknown"),
                        "Opening": headers.get("Opening", "Unknown"),
                        "Move Number": move_number,
                        "Player": headers.get("White", "Unknown") if move_number % 2 == 1 else headers.get("Black", "Unknown"),
                        "Color": "White" if move_number % 2 == 1 else "Black",
                        "Blunder Move": node.board().san(next_node.move),
                        "Better Move": better_move,
                        "Comment": comment,
                        "Evaluation Before": eval_score_prev,
                        "Evaluation After": eval_score,
                        "Evaluation Difference": eval_diff,
                        "Position Before Blunder": before_blunder,
                        "Position After Blunder": after_blunder,
                    }
                    blunders.append(blunder_info)

                node = next_node

    return blunders

def process_pgn_files(directory_path):
    all_blunders = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".pgn"):
            file_path = os.path.join(directory_path, file_name)
            blunders = extract_blunders(file_path)
            all_blunders.extend(blunders)

    return all_blunders

def save_to_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    pgn_directory_path = "Analysis_pgns_small"
    json_output_path = "blunders12321.json"

    all_blunders = process_pgn_files(pgn_directory_path)
    save_to_json(all_blunders, json_output_path)
    print(f"Blunders have been saved to {json_output_path}")

Blunders have been saved to blunders12321.json


In [11]:
import os
import re
import json
import chess.pgn
from concurrent.futures import ProcessPoolExecutor, as_completed

def parse_pgn(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def extract_blunders(pgn_file):
    blunders = []

    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            headers = game.headers
            node = game
            move_number = 0

            while node.variations:
                next_node = node.variation(0)
                move_number += 1

                comment = next_node.comment
                eval_match = re.search(r'\[%eval (-?\d+\.\d+)\]', comment)
                eval_match_prev = re.search(r'\[%eval (-?\d+\.\d+)\]', node.comment)
                if eval_match and eval_match_prev:
                    eval_score = float(eval_match.group(1))
                    eval_score_prev = float(eval_match_prev.group(1))
                else:
                    eval_score = None
                    eval_score_prev = None

                if re.search(r'\b[Bb]lunder\b', comment):
                    before_blunder = node.board().fen()
                    after_blunder = next_node.board().fen()

                    better_move_match = re.search(r'Blunder\.\s*(.*?)\s*was best\.', comment)
                    if better_move_match:
                        better_move = better_move_match.group(1)
                    else:
                        better_move = None

                    eval_diff = None
                    if eval_score is not None and eval_score_prev is not None:
                        eval_diff = eval_score - eval_score_prev

                    blunder_info = {
                        "Event": headers.get("Event", "Unknown"),
                        "Site": headers.get("Site", "Unknown"),
                        "Date": headers.get("Date", "Unknown"),
                        "Round": headers.get("Round", "Unknown"),
                        "White Player": headers.get("White", "Unknown"),
                        "Black Player": headers.get("Black", "Unknown"),
                        "Result": headers.get("Result", "Unknown"),
                        "UTCDate": headers.get("UTCDate", "Unknown"),
                        "UTCTime": headers.get("UTCTime", "Unknown"),
                        "Variant": headers.get("Variant", "Standard"),
                        "ECO": headers.get("ECO", "Unknown"),
                        "Opening": headers.get("Opening", "Unknown"),
                        "Move Number": move_number,
                        "Player": headers.get("White", "Unknown") if move_number % 2 == 1 else headers.get("Black", "Unknown"),
                        "Color": "White" if move_number % 2 == 1 else "Black",
                        "Blunder Move": node.board().san(next_node.move),
                        "Better Move": better_move,
                        "Comment": comment,
                        "Evaluation Before": eval_score_prev,
                        "Evaluation After": eval_score,
                        "Evaluation Difference": eval_diff,
                        "Position Before Blunder": before_blunder,
                        "Position After Blunder": after_blunder,
                    }
                    blunders.append(blunder_info)

                node = next_node

    return blunders

def process_pgn_files_parallel(pgn_directory, num_workers=4):
    pgn_files = [os.path.join(pgn_directory, file) for file in os.listdir(pgn_directory) if file.endswith(".pgn")]
    all_blunders = []

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = {executor.submit(extract_blunders, pgn_file): pgn_file for pgn_file in pgn_files}

        for future in as_completed(futures):
            pgn_file = futures[future]
            try:
                blunders = future.result()
                all_blunders.extend(blunders)
            except Exception as e:
                print(f"Error processing {pgn_file}: {e}")

    return all_blunders

def save_to_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    pgn_directory_path = "Analysis_pgns_small"
    json_output_path = "blunders1232123.json"

    all_blunders = process_pgn_files_parallel(pgn_directory_path)
    save_to_json(all_blunders, json_output_path)
    print(f"Blunders have been saved to {json_output_path}")

Error processing Analysis_pgns_small/0udqWQdc---1910 Lasker vs. Janowski.pgn: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing Analysis_pgns_small/0KNdpNQd---1910 Lasker vs. Schlechter.pgn: A process in the process pool was terminated abruptly while the future was running or pending.
Blunders have been saved to blunders1232123.json


Process SpawnProcess-9:
Traceback (most recent call last):
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'extract_blunders' on <module '__main__' (built-in)>
Process SpawnProcess-12:
Traceback (most recent call last):
  File "/Users/alankritverma/miniconda3/envs/CSSLAB/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/alankritverma/miniconda3/envs/CS